In [1]:
import gensim
import re
filename="/mnt/data/test_data/wiki_ko.txt"

In [2]:
ex_hangul = re.compile('[^ ㄱ-ㅣ가-힣|.]+') # 한글과 띄어쓰기를 제외한 모든 글자
ex_split_sentence = ['',' ']
def map_hangul(str):  #한글만 추출하는 메소드
    str = str.replace('_',' ')
    str = ex_hangul.sub('',str)
    str = str.replace('  ',' ')
    return str

def split_sentence(str): #마침표로 문장을 나누고, 빈 문장을 제거하는 메소드
    sentence_arr = str.split('.')
    return list(filter(lambda x: x not in ex_split_sentence, sentence_arr))
    

In [4]:
#문장 전처리
sentence_arr = []
with open(filename, "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        tmp_sentences = split_sentence(map_hangul(line))
        if len(tmp_sentences) > 0:
            sentence_arr.append(tmp_sentences)
print(sentence_arr[:5])

[[' 동화사'], ['동화사'], ['동화사는 대한민국 대구광역시 동구 도학동에 있는 절이다', ' 대한불교 조계종 경북 대 본산 중의 하나이자 대한불교 조계종 제교구 본사이다'], ['동화사사적비년에 따르면 신라 소지왕 년년에 극달 화상이 유가사라는 이름으로 처음 지은 것을 흥덕왕 년년 심지조사가 중창하면서 지금의 이름으로 바꾸었다고 한다', ' 동화사라는 이름은 중창할 당시 겨울인데도 오동나무가 꽃을 피운 것을 보고 상서로운 징조로 여긴 심지조사가 붙인 이름이라고 한다'], ['삼국유사세기에는 진표로부터 불골간자를 이어받은 영심이 다시 심지조사에게 간자 두 개를 전했고 심지조사가 그것을 봉안할 땅을 찾고자 중악의 두 신과 함께 산꼭대기에 올라가 서쪽을 향해 던지니 간자가 곧 바람에 날려 날아갔고 간자가 떨어진 곳이 동화사 참당의 북쪽에 있는 작은 우물이었다', ' 그리고 심지조사는 그곳에 강당을 지어 간자를 봉안했다고 한다']]


In [26]:
#명사 추출
from konlpy.tag import Twitter
MINIMUM_SENTENCE_LENGTH = 3

twitter = Twitter()
sentence_noun_arr = []

for sentence in sentence_arr:
    tmp_noun_arr = twitter.nouns(sentence[0])
    if len(tmp_noun_arr) > MINIMUM_SENTENCE_LENGTH:
        sentence_noun_arr.append(tmp_noun_arr)
print(sentence_noun_arr[:5])

[['동화사', '대한민국', '대구광역시', '동구', '도학동', '절'], ['동화사', '사적', '비년', '신라', '소지왕', '년년', '달', '화상', '유가사', '이름', '처음', '지은', '것', '흥덕왕', '년년', '심지', '조사', '중창', '지금', '이름'], ['삼국유사', '세기', '진표', '로부터', '불골', '간자', '영심', '다시', '심지', '조사', '간자', '두', '개', '심지', '조사', '그것', '봉안', '땅', '중', '악의', '두', '신', '산꼭대기', '서쪽', '간자', '곧', '바람', '간자', '곳', '동화사', '당', '북쪽', '우물'], ['경문왕', '년년', '동화사', '딸린', '비', '암', '층', '석탑', '석조', '비로자나불', '경문왕', '자신', '명의', '조성'], ['동화사', '신라', '말기', '고려', '후백제', '현장', '견훤', '신라', '공격', '때', '신라', '구원', '요청', '왕건', '거', '온', '만', '명의', '군사', '곳', '동수', '동화사', '주둔', '후백제', '전투', '패']]


In [88]:
#word2vec 학습

# model = gensim.models.Word2Vec(size=300, window=5, min_count=50, workers=2)
# model.build_vocab(sentence_noun_arr)
# model.train(sentence_noun_arr)
# model.save('dict_data/w2v_model_wiki_kor')

In [89]:
print(model.wv.similarity('미국', '일본'))
print(model.wv.most_similar(positive=['문재인', '박근혜'], negative=['노무현']))
print(model.wv.doesnt_match(twitter.nouns('박근혜는 가나다를 좋아한다')))
# model.score([twitter.nouns('박근혜는 가나다를 좋아한다')])

0.237595433168
[('정몽준', 0.8667929768562317), ('안철수', 0.842223584651947), ('신한국당', 0.8410040140151978), ('새누리당', 0.8406327962875366), ('한나라당', 0.8380944728851318), ('정동영', 0.8352590203285217), ('이회창', 0.8274019360542297), ('우리당', 0.8226585388183594), ('서울시장', 0.8213465809822083), ('박원순', 0.8142303228378296)]
박근혜


In [95]:
new_model = gensim.models.Word2Vec.load('dict_data/w2v_model_wiki_kor')

In [96]:
print(new_model.wv.similarity('미국', '일본'))

0.315325904718
